# **DSCI 100 Group Project**
Authors: 
- Munn Chai
- Deep Shah
- Your name here
- Your name here

### **Introduction**
info needed:
- context, background info about topic
- state question clearly
- identify dataset that will be used

### **Preliminary Exploratory Data Analysis**
info needed: 
- read dataset into R
- wrangle data into tidy data
- using Training data only, summarize the data in one table. An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
- Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

### **Methods**
info needed: 
- explain how you will conduct your data analysis, and which rows and columns you will use
- describe one way you will visualize the results

### **Expected Outcomes and Significance**
info needed: 
- what do you expect to find?
- what impact do these findings have? 
- what future questions could these findings lead to? 